# Live Plotting

Real-time visualization of sweep data.

## Setup

```bash
# Terminal: Enable live plotting
stanza live-plot enable --backend inline  # Plots in cells
# OR
stanza live-plot enable                   # Plots in browser
```

Then run cells below. DataLogger auto-detects configuration.

In [ ]:
import logging
import tempfile
import time

import numpy as np

from stanza.logger.data_logger import DataLogger

# Configure logging to see INFO messages
logging.basicConfig(level=logging.INFO, format="%(name)s - %(levelname)s - %(message)s")

In [ ]:
tmpdir = tempfile.mkdtemp()
logger = DataLogger("basic", tmpdir)

## Example 1: Basic 1D Plot

In [ ]:
session = logger.create_session()

# Log data
x = np.linspace(0, 10, 50)
y = np.sin(x)
session.log_sweep(name="sine", x_data=x, y_data=y, x_label="Time", y_label="Amplitude")
session.flush()

session.close()

## Example 2: Streaming Data

Use `sweep()` context to stream data point-by-point. Each `append()` updates the plot immediately.

In [ ]:
tmpdir = tempfile.mkdtemp()
logger = DataLogger("streaming", tmpdir)
session = logger.create_session()

with session.sweep("decay", x_label="Time (s)", y_label="Signal") as s:
    for i in range(50):
        t = i * 0.1
        signal = np.exp(-t / 2)
        s.append([t], [signal])
        time.sleep(0.05)

session.close()

## Example 3: 2D Heatmap

Use list for `x_label` to create heatmap. Perfect for stability diagrams.

In [ ]:
tmpdir = tempfile.mkdtemp()
logger = DataLogger("2d", tmpdir)
session = logger.create_session()

with session.sweep(
    "heatmap",
    x_label=["Gate 1 (V)", "Gate 2 (V)"],  # List = 2D
    y_label="Current (nA)",
) as s:
    for v1 in np.linspace(-1, 1, 30):
        for v2 in np.linspace(-1, 1, 30):
            current = np.sin(3 * v1) * np.cos(3 * v2)
            s.append([v1, v2], [current])
            time.sleep(0.01)

session.close()

## Example 4: Charge Stability Diagram

Simulate live data acquisition from a real measurement. This example loads pre-recorded charge stability diagram data and streams it point-by-point to demonstrate real-time plotting of quantum device characterization.

In [ ]:
import h5py

# Load charge stability diagram data
data_path = "example_data/charge_stability_diagram.h5"
with h5py.File(data_path, "r") as f:
    P1_values = f["P1_values"][()]
    P2_values = f["P2_values"][()]
    sensor_current = f["sensor_current"][()]

tmpdir = tempfile.mkdtemp()
logger = DataLogger("csd", tmpdir)
session = logger.create_session()

with session.sweep(
    "charge_stability_diagram",
    x_label=["P1 (V)", "P2 (V)"],  # Gate voltages
    y_label="Sensor Current (nA)",
) as s:
    # Simulate live data acquisition by iterating through the 2D grid
    for i, p1 in enumerate(P1_values):
        for j, p2 in enumerate(P2_values):
            current = sensor_current[i, j] * 1e9  # Convert to nA
            s.append([p1, p2], [current])
            time.sleep(0.001)  # Simulate measurement time

session.close()

## API Summary

**Two methods:**
1. `session.log_sweep()` + `flush()` - Log complete datasets
2. `session.sweep()` context - Stream point-by-point (recommended)

**1D plots:** `x_label="string"`, append `([x], [y])`  
**2D heatmaps:** `x_label=["X", "Y"]`, append `([x, y], [value])`

**Backends:**
- `inline` - Plots in cells (requires `jupyter_bokeh`)
- `server` - Plots in browser at `http://localhost:5006`